МФТИ ФИВТ, Машинное обучение, Никита Волков

# Ранжирование

* Дедлайн **14 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями.
* Кагл: https://inclass.kaggle.com/c/ml-mipt-ranking4
* Инвайт: https://kaggle.com/join/4uthnui9sndi

### Небольшая предыстория

Внезапно очутившись на 4 курсе (как это произошло, вообще не понятно) студенты ФИВТа поняли, что времени стало совсем не хватать. Немного подумав, они решили, что во всем виноваты обезумевшие преподаватели курса Машинное обучение. Было проведено коллективное собрание, на котором было принято решение — попробовать договориться с преподавателями. Однако, все попытки примирения были отвергнуты. И тогда на очередном собрании (вместо выполнения очередного ДЗ ;)) было принято решение сделать вопросно-ответную систему для студентов. Например, она сможет четко отвечать на вопросы следующего типа ”Когда дедлайн по очередному огромному заданию?”, ”Что нужно сделать в задании?”, а вопрос ”Сколько суток нужно потратить, чтобы выполнить очередное ДЗ” даже никто и задавать не будет — никто их не делает, кроме самих преподавателей. А на вопрос ”Сможем ли мы переубедить Арсения Ашуху?” система ответит картинкой:

<img width=400 src="./pic.png">

Для каждого вопроса студенты смогли написать систему, которая подбирает наиболее подходящие ответы в интернете. А вот, как из упорядочить, они не знают, поскольку прогуляли лекцию по ранжированию. К счастью, было записано видео этой лекции, так что вы сможете им помочь.

А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

### Бейзлайн

In [31]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Считываем все запросы из test

In [147]:
with open('contest/queries_test.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

По очереди загружаем все документы, считая для каждого TF и сортируя для каждого запроса. Это и есть скор в бейзлайне.

In [2]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)):
        return False
    return True

In [124]:
with open('contest/queries_train.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

In [149]:
bad_links = list()
with open('contest/test.txt') as ft:
    ft.readline()
    for i, line in enumerate(ft):
        if i % 50 == 0:
            print(i)

        split_line = line.split(',')
        qid, docid, url = int(split_line[0]), int(split_line[1]), ','.join(split_line[2:])

        try:
            text = BeautifulSoup(urlopen(url, timeout=10).read(), 'html.parser').findAll(text=True)
            text = list(filter(visible, text))
            text = ' '.join(text)
            text = np.array(text.split(' '))
        except Exception as e:
            bad_links.append([qid, docid, url])
            print(qid, docid, url)

        np.save('contest/test/{}_{}.npz'.format(qid, docid), text)

0
103 6 http://tushkan.tv/news/tancy/2016-07-13-19014

103 8 http://86sch-mortka.edusite.ru/DswMedia/kakrazvit-navyikisamobladaniya.doc

50
100
150
200
109 23 http://4cheat.ru/showthread.php?t=141219

250
111 9 http://mail.kpk-user.ru/17243-pochemu-vse-begayut-po-ulicam-vo-vremya-igry-pokemon-go-poslednie-novosti-segodnya-26-iyulya-2016.html

112 21 http://nashi-de-ti.ru/s-kakogo-vozrasta-mozhno-davat-rebenku-moloko-I-kakoe

300
113 2 http://franklin100.ru/

114 2 http://poiskm.org/show/%D1%85%D0%B5%D0%BC%D0%B8%D1%81%D0%B8%D0%BD%D0%BA/%D0%BA%D0%B0%D0%BA-%D0%B2%D1%8B%D1%81%D0%BF%D0%B0%D1%82%D1%8C%D1%81%D1%8F-%D0%B7%D0%B0-30-%D0%BC%D0%B8%D0%BD%D1%83%D1%82

350
400
118 24 http://moya-semya.ru/index.php?Itemid=156\u0026catid=91:2011-06-17-11-03-00\u0026id=6573:mvs2015-31\u0026option=com_content\u0026view=article

450
500
550
123 12 http://ask.fm/rozhdennij_pod_solncem

600
128 2 http://moredogs.ru/pochemu-mysh-boitsya-koshki-a-koshka-boitsya-sobak-a-slon-boitsya-myshi.html

129 1 http://mo

In [11]:
with open('contest/train.txt') as ft:
    ft.readline()
    for i, line in enumerate(ft):
        if i % 50 == 0:
            print(i)

        split_line = line.split(',')
        rel, qid, docid, url = int(split_line[0]), int(split_line[1]), int(split_line[2]), ','.join(split_line[3:])

        try:
            text = BeautifulSoup(urlopen(url, timeout=10).read(), 'html.parser').findAll(text=True)
            text = list(filter(visible, text))
            text = ' '.join(text)
            text = np.array(text.split(' '))
        except Exception as e:
            bad_links.append([qid, docid, url])
            print(qid, docid, url)

        np.save('contest/train/{}_{}.npz'.format(qid, docid), text)

0
1 28 http://sonme.ru/rezult/?about-dream=%D0%97%D0%B0%D1%81%D0%BD%D1%83%D1%82%D1%8C%20%D0%B2%D0%BE%20%D1%81%D0%BD%D0%B5

50
100
150
200
250
300
16 14 http://funlol.ru/19484-ya-russkiy-okkupant.html

17 3 http://anekdot.kz/zagadki/kak-nazyvaetsya-boyazn-prihoda-santa-klausa/

350
400
23 13 http://seria-z.net/

24 5 http://joyreactor.cc/post/2497855

450
500
28 18 http://rihiqi.zlatoruki.ru/

550
30 23 http://durdom2.com/post.php?post=53

31 15 http://www.ljpoisk.ru/mama/2014/01/13304479.html

600
650
700
38 16 http://magiyaforum.ru/archive/index.php/t-763.html

39 9 http://anecdots.su/anecdot/16039

750
800
850
900
46 5 http://lady-shik.ga/mordkovich/

46 7 http://evgeniyvolodin.ga/goroda/

950
1000
1050
1100
54 8 http://logsi.ru/

1150
1200
57 10 http://moredogs.ru/kto-opredelyaet-chto-korm-dlya-sobak-ili-koshek-stal-eshhe-vkusnee.html

1250
59 24 http://muzogig.net/gig/%D1%82%D0%B5%D0%BF%D0%B5%D1%80%D1%8C%20%D1%8F%20%D1%81%D1%82%D0%B0%D0%BB%20%D0%B1%D0%BE%D0%B3%D0%B0%D1%82%D1%8B%D0%

In [150]:
very_bad_links = list()
for qid, docid, url in bad_links:
    for _ in range(10):
        try:
            text = BeautifulSoup(urlopen(url, timeout=20).read(), 'html.parser').findAll(text=True)
            text = list(filter(visible, text))
            text = ' '.join(text)
            text = np.array(text.split(' '))
            np.save('contest/test/{}_{}.npz'.format(qid, docid), text)

            #print(url, ' done')
            break
        except Exception as e:
            pass

    print(qid, docid, url, ' failed')
    very_bad_links.append([qid, docid, url])

103 6 http://tushkan.tv/news/tancy/2016-07-13-19014
  failed
103 8 http://86sch-mortka.edusite.ru/DswMedia/kakrazvit-navyikisamobladaniya.doc
  failed
109 23 http://4cheat.ru/showthread.php?t=141219
  failed
111 9 http://mail.kpk-user.ru/17243-pochemu-vse-begayut-po-ulicam-vo-vremya-igry-pokemon-go-poslednie-novosti-segodnya-26-iyulya-2016.html
  failed
112 21 http://nashi-de-ti.ru/s-kakogo-vozrasta-mozhno-davat-rebenku-moloko-I-kakoe
  failed
113 2 http://franklin100.ru/
  failed
114 2 http://poiskm.org/show/%D1%85%D0%B5%D0%BC%D0%B8%D1%81%D0%B8%D0%BD%D0%BA/%D0%BA%D0%B0%D0%BA-%D0%B2%D1%8B%D1%81%D0%BF%D0%B0%D1%82%D1%8C%D1%81%D1%8F-%D0%B7%D0%B0-30-%D0%BC%D0%B8%D0%BD%D1%83%D1%82
  failed
118 24 http://moya-semya.ru/index.php?Itemid=156\u0026catid=91:2011-06-17-11-03-00\u0026id=6573:mvs2015-31\u0026option=com_content\u0026view=article
  failed
123 12 http://ask.fm/rozhdennij_pod_solncem
  failed
128 2 http://moredogs.ru/pochemu-mysh-boitsya-koshki-a-koshka-boitsya-sobak-a-slon-boitsya-mysh

In [151]:
for qid, docid, url in very_bad_links:
    np.save('contest/test/{}_{}.npz'.format(qid, docid), np.array([]))

In [152]:
queries[:5]

['Почему нельзя неподошедшего тебе мужика просто аккуратненько вернуть обратно маме?',
 'Ты никогда не замечал, какие все вокруг оптимисты. когда проблема касается тебя?',
 'В какой момент нужно сказать привет когда навстречу идет знакомый?',
 'Кто знает, какие успокоительные лекарства принимает медведь в мультике про Машу? мне тоже такие надо.',
 'Таджики на уроке физкультуры по команде строимся,по строили новый дом .']

In [153]:
def calc_features(text, query, url):
    features = list()
    features.append(query[-1] == '?')
    try:
        pr_text = list(filter(lambda x: x != '\n', text))
        pr_text = list(filter(lambda x: x.isalpha() or x.isdigit(), pr_text))
    except:
        pr_text = text
    
    features.append(len(pr_text))
    features.append(len(query.split()))
    # print(text, query)

    query = query.replace(',', '')
    query = query.replace('-', ' ')
    query = query.lower().split()

    try:
        vectorizer = CountVectorizer().fit([' '.join(pr_text),])
        features.append(vectorizer.transform(query).mean())

        vectorizer2 = TfidfVectorizer().fit([' '.join(pr_text),])
        features.append(vectorizer2.transform(query).mean())
        
    except Exception as e:
        print(pr_text)
        features.append(0)
        features.append(0)
    
    features.append(query[0] == 'как')
    features.append(query[0] == 'если')
    features.append(query[0] == 'почему')
    features.append(query[0] == 'а')
    
    features.append('wiki' in url)
    features.append('forum' in url)
    features.append('ru' in url)
    features.append('ua' in url)
    features.append('otvet' in url)
    features.append('com' in url)
    features.append('vk.' in url)
    features.append('sprash' in url)
    features.append('mail' in url)
    features.append('youtube' in url)
    return features

In [141]:
qids, docids, relevs, feats = [], [], [], []

with open('contest/train.txt') as ft:
    with open('contest/features_train/feat1.txt', 'w') as fw:
        ft.readline()
        for i, line in enumerate(ft):
            #if i > 200:
            #    break
                #print(i)

            split_line = line.split(',')
            rel, qid, docid, url = int(split_line[0]), int(split_line[1]), int(split_line[2]), ','.join(split_line[3:])

            text = np.load('contest/train/{}_{}.npz.npy'.format(qid, docid))
            query = queries[qid-1]
        
            features = calc_features(text, query, url)
            feat_str = ' '.join(['{}:{}'.format(j+1, f) for j, f in enumerate(features)])
            
            qids.append(qid)
            docids.append(docid)
            relevs.append(rel)
            feats.append(features)

[]
[]
[]
[]
[]
[]
[]
[]
[]
['0', '0', '0', '2']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['\n' '\n' '\n' ...,
 'https://www.youtube.com/channel/UCNYVnuDr44zL49Gd2-KQNPw' '•••'
 '["musicclob.error","messaging","discussions","error","uncaught","notificationauth","videochat","videochatfp","share","pins","clickphotogetlink","musicusage","musicerror","mfe","photo","join","toolbarperformancenim","musicalpresents","present","messages","invitation","inline-comments","suggestionperformance","flash","searchsuggest","musageduration","dynamic-user-load","notifications","postingform","osn","omnicaching","musicrender","musicnoflash","vpl","vpal","search","reveal","poll","mtl","feed","mediatopic","appnotreaction","video","stt","ta","closeable_hook","mcdn","layerManager","mtlayer","appsstats","traceimg","mtplace","searchfb","jsfeatures","attach","sb","html5","require","toolbarperformancedisc","mt","asm","click","bday","amsg","push","giftFront","ad","ntf","banner","GoRec","GoSt

In [215]:
with open('contest/test.txt') as ft:
    with open('contest/test/feat1.txt', 'w') as fw:
        ft.readline()
        for i, line in enumerate(ft):
            #if i > 200:
            #    break
                #print(i)

            split_line = line.split(',')
            qid, docid, url = int(split_line[0]), int(split_line[1]), ','.join(split_line[2:])

            text = np.load('contest/test/{}_{}.npz.npy'.format(qid, docid))
            query = queries[qid-101]

            features = calc_features(text, query, url)
            feat_str = ' '.join(['{}:{}'.format(j+1, float(f)) for j, f in enumerate(features)])
            fw.write('{} qid:{} {}\n'.format(1, qid, feat_str))

['0', '0', '0', '0', '0', '0', '0', '0']
[]
[]
[]
[]
[]
[]
[]
[]
['B']
[]
['f']
[]
[]
[]
[]
[]
[]
['0', 'g']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['0', '0', '0', '0', '0', '0', '0']
[]
[]
[]
[]


In [156]:
def read_file(file_path, features_count):
    ''' Считывает дата-файл по адресу file_path, в котором есть не более features_count признаков.
    Возвращает список меток релевантности, id запросов и матрицу признаков'''
    
    relevs = []
    qids = []
    features = []
    
    with open(file_path) as data_file:
        for line in data_file:
            split_line = line.rstrip().split(' ')
            
            # print(split_line)
            # релевантность и id запроса

            relevs.append(split_line[0])
            qids.append(int(split_line[1].split(':')[1]))
            
            # признаки
            object_features = np.zeros(features_count, dtype=float)
            for feat in split_line[2:]:
                index, value = map(float, feat.split(':'))
                object_features[index] = value
            
            features.append(object_features)
    
    return relevs, qids, np.array(features)

def write(features, relevs, qids, file_path, index_begin, index_end):
    ''' Создает файл по адресу file_path, в который будут записаны релевантности relevs,
    номера запросов qids и признаки features с номера index_begin по index_end. '''
    
    with open(file_path, 'w') as f:
        for index_line in range(index_begin, index_end):
            f.write('{} qid:{}'.format(relevs[index_line], qids[index_line]))
            for i in range(features.shape[1]):
                f.write(' {}:{}'.format(i + 1, features[index_line, i]))
            f.write('\n')


def split_to_train_valid_test(relevs, qids, features, 
                              train_path, test_path,
                              train_size, test_size,
                              valid_path=None, valid_size=None):
    ''' Разбивает датасет на две или три части, и записывает их в файлы'''

    num_docs = len(qids)
    i_0 = 0
    
    qids = np.array(qids)
    is_new_query = qids[:-1] != qids[1:]  # True в тех позициях, в которых начинается новый запрос
    new_query_positions = np.arange(num_docs - 1)[is_new_query]  # Позиции, в которых начинается новый запрос

    # Ищем позицию, на которой заканчивается train
    allow_positions = new_query_positions > (train_size * num_docs)
    i_1 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

    # Ищем позицию, на которой заканчивается test
    allow_positions = new_query_positions > ((train_size + test_size) * num_docs)
    i_2 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

    write(features, relevs, qids, train_path, i_0, i_1)
    write(features, relevs, qids, test_path, i_1, i_2)

    if valid_path is not None:
        # Ищем позицию, на которой заканчивается valid
        allow_positions = new_query_positions > ((train_size + test_size + valid_size) * num_docs)
        i_3 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

        write(features, relevs, qids, valid_path, i_2, i_3)

In [225]:
split_to_train_valid_test(np.array(relevs), np.array(qids), np.array(feats),
                          'contest/features_train/train_train.txt', 
                          'contest/features_train/train_test.txt', 
                          0.9, 0.1)

In [226]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./contest/features_train/train_train.txt -test ./contest/features_train/train_test.txt -validate ./contest/features_train/train_valid.txt -ranker 6 -tree 20 -leaf 5 -metric2t NDCG@10 -save ./contest/LambdaMART_300.txt > ./contest/log_LambdaMART_300.txt

CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 906 ms


In [227]:
! java -jar RankLib-2.1-patched.jar -load ./contest/LambdaMART_300.txt -test ./contest/features_train/train_test.txt -metric2T NDCG@10


[+] General Parameters:
Model file:	./contest/LambdaMART_300.txt
Feature normalization: No
Test metric:	NDCG@10
Model:		LambdaMART
Reading feature file [./contest/features_train/train_test.txt]... [Done.]            
(7 ranked lists, 207 entries read)
NDCG@10 on test data: 0.7237


In [228]:
! java -jar RankLib-2.1-patched.jar -load ./contest/LambdaMART_300.txt -rank ./contest/test/feat1.txt -score ./contest/myscorefile.txt


[+] General Parameters:
Model file:	./contest/LambdaMART_300.txt
Feature normalization: No
Model:		LambdaMART
Reading feature file [./contest/test/feat1.txt]... [Done.]            
(99 ranked lists, 2383 entries read)


In [229]:
with open('contest/myscorefile.txt') as f:
    solution = f.readlines()

In [231]:
solution = list(map(lambda x: x.split('\t'), solution))

In [234]:
solution = list(map(lambda x: [int(x[0]), int(x[1]), float(x[2])], solution))

In [235]:
solution = np.array(solution)

In [248]:
with open('contest/solution2.txt', 'w') as fw:
    fw.write('QueryId,DocumentId\n')
    
    for qid in range(101, 201):
        query = list(filter(lambda x: x[0] == qid, solution))
        query.sort(key=lambda x: -x[2])

        for q, doc, rel in query:
            fw.write('{},{}\n'.format(qid, int(doc)))

Кратко: из текста страницы оставила только слова и цифры. Из запроса убала знаки препинания, все сделала прописными буквами.  Для каждой страницы посчитала следующие фичи (query - запрос, массив слов; url - ссылка; text - текст страницы):
<ul type="circle"> 
<li> средняя частота слов из text $\cap$ query</li>
<li> средний TF-IDF слов из  text $\cap$ query </li>
<li> query[0] == 'как' </li>
<li>query[0] == 'если'</li>
<li>query[0] == 'почему'</li>
<li>query[0] == 'а'</li>
    
<li>'wiki' in url</li>
<li>'forum' in url</li>
<li>'ru' in url</li>
<li>'ua' in url</li>
<li>'otvet' in url</li>
<li>'com' in url</li>
<li>'vk.' in url</li>
<li>'sprash' in url</li>
<li>'mail' in url</li>
<li>'youtube' in url</li>
</ul>

Запустила лямбда март на 10 деревьях.